<a href="https://colab.research.google.com/github/swastikakhan/FaceRecognition_GenderClassification/blob/main/TaskA_98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Mount Google Drive and extract dataset
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive')

# Extract the dataset
zip_path = '/content/drive/MyDrive/dataset2.zip'
extract_path = '/content/'

print("Extracting dataset...")
print(f"Zip file exists: {os.path.exists(zip_path)}")

if not os.path.exists(zip_path):
    print("ERROR: dataset2.zip not found in MyDrive!")
    print("Contents of MyDrive:")
    print(os.listdir('/content/drive/MyDrive/'))
    exit()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    print("Files in zip:")
    for file in zip_ref.namelist()[:10]:  # Show first 10 files
        print(f"  {file}")
    if len(zip_ref.namelist()) > 10:
        print(f"  ... and {len(zip_ref.namelist()) - 10} more files")

    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

# Check what was extracted
print("\nChecking extracted contents:")
if os.path.exists('/content/Comsys_Hackathon5'):
    print("Found Comsys_Hackathon5 folder")
    print("Contents:", os.listdir('/content/Comsys_Hackathon5'))

    if os.path.exists('/content/Comsys_Hackathon5/Task_B'):
        print("Found Task_B folder")
        print("Contents:", os.listdir('/content/Comsys_Hackathon5/Task_B'))
    else:
        print("Task_B folder not found")
else:
    print("Comsys_Hackathon5 folder not found")
    print("Contents of /content/:")
    for item in os.listdir('/content/'):
        print(f"  {item}")
        if os.path.isdir(f'/content/{item}') and 'Comsys' in item:
            print(f"    Contents of {item}:", os.listdir(f'/content/{item}'))

# 2. Install dependencies
!pip install facenet-pytorch scikit-learn tqdm

# 3. Imports
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from PIL import Image
import numpy as np
import torch

# 4. Setup models with better parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# More aggressive face detection parameters
mtcnn = MTCNN(
    image_size=160,
    margin=0,
    min_face_size=20,  # Smaller minimum face size
    thresholds=[0.4, 0.5, 0.5],  # More lenient thresholds
    factor=0.709,  # Default scaling factor
    post_process=True,
    device=device
)

facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# 5. Helper: Extract face embedding with better preprocessing
def get_embedding(img_path):
    try:
        img = Image.open(img_path).convert('RGB')

        # Check image size and resize if too small
        if img.size[0] < 160 or img.size[1] < 160:
            # Resize smaller images to minimum size
            img = img.resize((max(160, img.size[0]), max(160, img.size[1])), Image.Resampling.LANCZOS)

        # Try multiple detection strategies
        face = None

        # Strategy 1: Standard detection
        face = mtcnn(img)

        # Strategy 2: More lenient detection for difficult images
        if face is None:
            mtcnn_lenient = MTCNN(
                image_size=160,
                margin=0,
                min_face_size=15,  # Even smaller
                thresholds=[0.3, 0.4, 0.4],  # Very lenient
                device=device
            )
            face = mtcnn_lenient(img)

        # Strategy 3: Try with different image enhancements
        if face is None:
            # Enhance contrast for low-light/foggy images
            from PIL import ImageEnhance
            enhancer = ImageEnhance.Contrast(img)
            enhanced_img = enhancer.enhance(2.0)  # Increase contrast
            face = mtcnn(enhanced_img)

        if face is None:
            return None

        # Normalize the face embedding (L2 normalization)
        with torch.no_grad():
            emb = facenet(face.unsqueeze(0).to(device))
            emb = emb.squeeze().cpu().numpy()
            # L2 normalize the embedding
            emb = emb / np.linalg.norm(emb)

        return emb
    except Exception as e:
        print(f"      Error processing {img_path}: {e}")
        return None

# 6. Prepare dataset with better error handling
def load_dataset(folder):
    X, y = [], []
    print(f"Loading dataset from: {folder}")

    if not os.path.exists(folder):
        print(f"Error: Folder {folder} does not exist!")
        return np.array([]), np.array([])

    total_images = 0
    total_faces = 0
    failed_folders = []

    person_folders = sorted(os.listdir(folder))
    print(f"Found {len(person_folders)} person folders")

    for person in person_folders:
        person_dir = os.path.join(folder, person)
        if not os.path.isdir(person_dir):
            continue

        person_count = 0
        person_images = 0

        # Process all images in the person directory and subdirectories
        for root, dirs, files in os.walk(person_dir):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(root, file)
                    person_images += 1
                    total_images += 1

                    emb = get_embedding(img_path)
                    if emb is not None:
                        X.append(emb)
                        y.append(person)
                        person_count += 1
                        total_faces += 1

        if person_count == 0:
            failed_folders.append(person)
        elif person_count > 0:
            print(f"  {person}: {person_count}/{person_images} faces detected")

    print(f"Total: {total_faces} faces detected out of {total_images} images")
    print(f"Successfully processed: {len(person_folders) - len(failed_folders)}/{len(person_folders)} folders")

    if len(failed_folders) > 0:
        print(f"Failed folders (no faces detected): {len(failed_folders)}")
        if len(failed_folders) <= 10:
            print(f"  Failed: {failed_folders}")

    if len(X) == 0:
        print("ERROR: No face embeddings extracted!")
        return np.array([]), np.array([])

    return np.array(X), np.array(y)

# 7. Set dataset paths - Find the correct paths dynamically
def find_dataset_paths():
    base_paths = [
        '/content/Comsys_Hackathon5/Task_B',
        '/content/dataset2/Comsys_Hackathon5/Task_B',
        '/content/Task_B'
    ]

    # Check common extraction locations
    for base_path in base_paths:
        train_path = os.path.join(base_path, 'train')
        val_path = os.path.join(base_path, 'val')

        if os.path.exists(train_path) and os.path.exists(val_path):
            return train_path, val_path

    # If not found, search recursively
    for root, dirs, files in os.walk('/content/'):
        if 'train' in dirs and 'val' in dirs:
            # Check if this looks like our dataset structure
            train_path = os.path.join(root, 'train')
            val_path = os.path.join(root, 'val')

            # Quick check - should have subdirectories (person folders)
            if os.path.exists(train_path) and os.path.exists(val_path):
                train_subdirs = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
                val_subdirs = [d for d in os.listdir(val_path) if os.path.isdir(os.path.join(val_path, d))]

                if len(train_subdirs) > 0 and len(val_subdirs) > 0:
                    return train_path, val_path

    return None, None

train_dir, val_dir = find_dataset_paths()

if train_dir is None or val_dir is None:
    print("ERROR: Could not find train and val directories!")
    print("Please check the dataset structure.")
    exit()

print(f"Found dataset paths:")
print(f"Train dir: {train_dir}")
print(f"Val dir: {val_dir}")

# Verify paths exist
print(f"Train dir exists: {os.path.exists(train_dir)}")
print(f"Val dir exists: {os.path.exists(val_dir)}")

if os.path.exists(train_dir):
    print(f"Train dir contents: {os.listdir(train_dir)}")
if os.path.exists(val_dir):
    print(f"Val dir contents: {os.listdir(val_dir)}")

# 8. Load datasets
print("Extracting train embeddings...")
X_train, y_train = load_dataset(train_dir)
print(f"Train dataset: {X_train.shape[0]} samples")

print("Extracting val embeddings...")
X_val, y_val = load_dataset(val_dir)
print(f"Validation dataset: {X_val.shape[0]} samples")

# Check if we have enough data
if X_train.shape[0] == 0:
    print("ERROR: No training samples found!")
    print("Possible issues:")
    print("1. No faces detected in training images")
    print("2. Wrong dataset structure")
    print("3. Image format issues")

    # Let's check what's actually in the directories
    print("\nDebugging - checking actual directory contents:")
    if os.path.exists(train_dir):
        for person in os.listdir(train_dir):
            person_dir = os.path.join(train_dir, person)
            if os.path.isdir(person_dir):
                files = [f for f in os.listdir(person_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                print(f"  {person}: {len(files)} image files")
                if len(files) > 0:
                    print(f"    Sample files: {files[:3]}")  # Show first 3 files

                    # Try to process one image manually for debugging
                    sample_img = os.path.join(person_dir, files[0])
                    print(f"    Testing image: {sample_img}")
                    try:
                        img = Image.open(sample_img).convert('RGB')
                        print(f"    Image size: {img.size}")
                        face = mtcnn(img)
                        if face is None:
                            print("    No face detected - image might be too small, blurry, or face not clearly visible")
                        else:
                            print(f"    Face tensor shape: {face.shape}")
                    except Exception as e:
                        print(f"    Error processing image: {e}")

                # Check distortion folder
                distortion_dir = os.path.join(person_dir, 'distortion')
                if os.path.exists(distortion_dir):
                    distortion_files = [f for f in os.listdir(distortion_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                    print(f"    Distortion folder: {len(distortion_files)} files")
                    if len(distortion_files) > 0:
                        print(f"      Sample distortion files: {distortion_files[:3]}")

    exit()

if X_val.shape[0] == 0:
    print("ERROR: No validation samples found!")
    exit()

# 9. Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

print(f"Number of classes: {len(le.classes_)}")
print(f"Classes: {le.classes_}")

# 10. Train multiple models and choose the best
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import joblib

print("Training multiple models...")

# Prepare data
print(f"Final dataset shapes: X_train={X_train.shape}, X_val={X_val.shape}")
print(f"Number of classes: {len(le.classes_)}")

# Model 1: Linear SVM with class balancing
print("\n1. Training Linear SVM (balanced)...")
svm_linear = SVC(kernel='linear', probability=True, class_weight='balanced', C=1.0)
svm_linear.fit(X_train, y_train_enc)
y_pred_svm_linear = svm_linear.predict(X_val)
acc_svm_linear = accuracy_score(y_val_enc, y_pred_svm_linear)
print(f"   Linear SVM Accuracy: {acc_svm_linear:.4f}")

# Model 2: RBF SVM with GridSearch
print("\n2. Training RBF SVM with GridSearch...")
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'class_weight': ['balanced', None]
}
svm_rbf = GridSearchCV(
    SVC(kernel='rbf', probability=True),
    param_grid,
    cv=min(5, len(np.unique(y_train_enc))),  # Use fewer folds if few classes
    scoring='accuracy',
    n_jobs=-1
)
svm_rbf.fit(X_train, y_train_enc)
y_pred_svm_rbf = svm_rbf.predict(X_val)
acc_svm_rbf = accuracy_score(y_val_enc, y_pred_svm_rbf)
print(f"   RBF SVM Accuracy: {acc_svm_rbf:.4f}")
print(f"   Best parameters: {svm_rbf.best_params_}")

# Model 3: Random Forest
print("\n3. Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train_enc)
y_pred_rf = rf.predict(X_val)
acc_rf = accuracy_score(y_val_enc, y_pred_rf)
print(f"   Random Forest Accuracy: {acc_rf:.4f}")

# Model 4: Neural Network (MLP)
print("\n4. Training Neural Network...")
mlp = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='adaptive',
    max_iter=500,
    random_state=42
)
mlp.fit(X_train, y_train_enc)
y_pred_mlp = mlp.predict(X_val)
acc_mlp = accuracy_score(y_val_enc, y_pred_mlp)
print(f"   Neural Network Accuracy: {acc_mlp:.4f}")

# Choose the best model
models = {
    'Linear SVM': (svm_linear, y_pred_svm_linear, acc_svm_linear),
    'RBF SVM': (svm_rbf, y_pred_svm_rbf, acc_svm_rbf),
    'Random Forest': (rf, y_pred_rf, acc_rf),
    'Neural Network': (mlp, y_pred_mlp, acc_mlp)
}

best_model_name = max(models.keys(), key=lambda x: models[x][2])
best_model, best_predictions, best_accuracy = models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name} (Accuracy: {best_accuracy:.4f})")

# Show detailed results for the best model
print(f"\nDetailed Classification Report for {best_model_name}:")
print(classification_report(y_val_enc, best_predictions, target_names=le.classes_))

print(f"\nFinal Metrics:")
print(f"Accuracy: {accuracy_score(y_val_enc, best_predictions):.4f}")
print(f"Precision: {precision_score(y_val_enc, best_predictions, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_val_enc, best_predictions, average='weighted'):.4f}")
print(f"F1-score: {f1_score(y_val_enc, best_predictions, average='weighted'):.4f}")

# Save the best model
joblib.dump(best_model, '/content/best_face_recognition_model.pkl')
joblib.dump(le, '/content/label_encoder.pkl')
print(f"\nBest model ({best_model_name}) and label encoder saved!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracting dataset...
Zip file exists: True
Files in zip:
  Comys_Hackathon5/
  Comys_Hackathon5/Task_B/
  Comys_Hackathon5/Task_A/
  Comys_Hackathon5/Task_B/val/
  Comys_Hackathon5/Task_B/train/
  Comys_Hackathon5/Task_A/val/
  Comys_Hackathon5/Task_A/train/
  Comys_Hackathon5/Task_B/val/Jennifer_Keller/
  Comys_Hackathon5/Task_B/val/Jennifer_Keller/Jennifer_Keller_0004.jpg
  Comys_Hackathon5/Task_B/val/Jennifer_Keller/Jennifer_Keller_0003.jpg
  ... and 23387 more files
Dataset extracted successfully!

Checking extracted contents:
Comsys_Hackathon5 folder not found
Contents of /content/:
  .config
  drive
  face_recognition_svm.pkl
  label_encoder.pkl
  Comys_Hackathon5
  sample_data
Using device: cuda
Found dataset paths:
Train dir: /content/Comys_Hackathon5/Task_A/train
Val dir: /content/Comys_Hackathon5/Task_A/val
Train dir exists: True
Val dir exists: Tr